The following file tries to predict the w2v embeddings
from the EEG data.

First I'll try to predict using the raw data (without averaging).

In [1]:
import sklearn
import pandas as pd
import numpy as np
import pickle
import gensim
from numpy import savez_compressed
from numpy import load
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
# from sklearn.svm import LinearSVR
# from sklearn.svm import SVR
# from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [4]:
readys_path = "Z:\\Jenn\\ml_df_readys.pkl"

# with open(pkl_path, 'rb') as f:
f = open(readys_path, 'rb')
readys_data = pickle.load(f)
f.close()


In [7]:
# sktime_path = "Z:\\Jenn\\ml_df_sktime.pkl"
#
# # with open(pkl_path, 'rb') as f:
# f1 = open(sktime_path, 'rb')
# sktime_data = pickle.load(f1)
# f1.close()

In [4]:
# No need to run this cell.
labels_mapping = {0:'baby', 1:'bear', 2:'bird', 3: 'bunny',
                  4:'cat', 5 : 'dog', 6: 'duck', 7: 'mom',
                  8: 'banana', 9: 'bottle', 10: 'cookie',
                  11: 'cracker', 12: 'cup', 13: 'juice',
                  14: 'milk', 15: 'spoon'}

In [15]:
# No need to run this cell.
# Extract the word labels first and store it locally.
def create_w2v_embeds():
    word_labels = readys_data['label'].values
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
    # First obtain all the embeddings for the words in the labels mapping.
    w2v_label_embeds = {}
    for key in labels_mapping:
        w2v_label_embeds[key] = model[labels_mapping[key]]
    all_embeds = []
    for label in word_labels:
        all_embeds.append(w2v_label_embeds[int(label)])
    savez_compressed('w2v_embeds/all_w2v_embeds.npz', all_embeds)


In [6]:
eeg_features = readys_data.iloc[:,:18000].values
w2v_embeds_loaded = load('w2v_embeds/all_w2v_embeds.npz')
w2v_embeds = w2v_embeds_loaded['arr_0']

In [ ]:
# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(eeg_features, w2v_embeds, test_size=0.1)
print(X_train.shape)
print(y_train.shape)
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
# X_train = X_train.reshape(X_train.shape[0],-1)
# y_train = y_train.reshape(y_train.shape[0],300)
ridge = DecisionTreeRegressor()
ridge.fit(X_train, y_train)
preds = ridge.predict(X_test)

points = 0
total_points = 0
for i in range(preds.shape[0]-1):
    s_i = y_test[i]
    s_j = y_test[i+1]
    s_i_pred = preds[i]
    s_j_pred = preds[i+1]
    dsii = cosine_similarity([s_i], [s_i_pred])
    dsjj = cosine_similarity([s_j], [s_j_pred])
    dsij = cosine_similarity([s_i], [s_j_pred])
    dsji = cosine_similarity([s_j], [s_i_pred])
    if (dsii + dsjj) >= (dsij + dsji):
        points += 1
    total_points += 1
print( points / total_points)


(1553, 18000)
(1553, 300)
